In [61]:
import requests
import pandas as pd 
# Replace YOUR_API_KEY with your actual API key
api_key = 'xD'
number_of_rows = 400

In [17]:
colmn_rename = {
    'index': 'date',
    '1. open':'open',
    '2. high':'high',
    '3. low' : 'low',	
    '4. close' : 'close',
    '5. adjusted close' :	'adjusted_close',
    '6. volume'	: 'volume',
    '7. dividend amount'	: 'dividend_amount',
    '8. split coefficient' : 'split_coefficient'
}

In [3]:
symbol = 'IBM'

# Construct the API endpoint URL
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={api_key}'

# Make a GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Get the close data from the response JSON
    data = response.json()['Time Series (Daily)']
    close_data = {date: float(values['4. close']) for date, values in data.items()}
    print(len(close_data),type(data))
    # print(close_data)
else:
    print(f'Request failed with status code {response.status_code} for {symbol}')


5878 <class 'dict'>


In [56]:
df_tmp = pd.DataFrame(data) \
    .T.reset_index() \
    .rename_axis('index', axis='columns') \
    .rename(columns = colmn_rename) 
#df_tmp.head()
cols = list(df_tmp.columns)
cols.remove('date')
df_tmp[cols] = df_tmp[cols].apply(pd.to_numeric, errors='coerce')
df_tmp['date'] = df_tmp['date'].apply(pd.to_datetime,format='%Y-%m-%d') 
df_tmp['close_percent_change'] = (df_tmp.loc[::-1].close.pct_change() * 100)
df_tmp.dtypes

index
date                    datetime64[ns]
open                           float64
high                           float64
low                            float64
close                          float64
adjusted_close                 float64
volume                           int64
dividend_amount                float64
split_coefficient              float64
close_percent_change           float64
dtype: object

In [67]:
df_fresh = df_tmp.iloc[:number_of_rows]
# df_fresh = df_tmp.iloc[:number_of_rows]
df2_old = df_tmp.tail(-number_of_rows)
df2_old

index,date,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,close_percent_change
400,2021-08-09,142.20,142.495,140.97,141.25,125.312363,4904065,1.64,1.0,-1.970990
401,2021-08-06,143.00,144.390,142.89,144.09,126.364745,3826835,0.00,1.0,0.924564
402,2021-08-05,143.03,143.410,142.22,142.77,125.207125,2757389,0.00,1.0,0.007005
403,2021-08-04,143.80,144.180,142.47,142.76,125.198355,2830079,0.00,1.0,-0.909280
404,2021-08-03,141.90,144.700,141.65,144.07,126.347205,4084724,0.00,1.0,1.873851
...,...,...,...,...,...,...,...,...,...,...
5873,1999-11-05,92.75,92.940,90.19,90.25,49.122781,13737600,0.00,1.0,-1.430756
5874,1999-11-04,94.44,94.440,90.00,91.56,49.835810,16697600,0.00,1.0,-2.977641
5875,1999-11-03,95.87,95.940,93.50,94.37,51.365284,10369100,0.00,1.0,-0.464086
5876,1999-11-02,96.75,96.810,93.69,94.81,51.604774,11105400,0.00,1.0,-2.005168


In [70]:
df_tmp.to_parquet(f'data/all/parquet/{symbol}.parquet')
df2_old.to_parquet(f'data/old/parquet/{symbol}.parquet')
df_fresh.to_csv(f'data/new/csv/{symbol}.csv.gz')